In [ ]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

def extract_features(images):
    features = []
    valid_images = []  # To store paths of successfully processed images
    for image in tqdm(images):
        try:
            img = load_img(image, target_size=(128, 128))  # Reduced image size
            img = np.array(img)  # Convert image to array
            features.append(img)  # Append to features list
            valid_images.append(image)  # Append valid image path
        except Exception as e:
            print(f"Skipping file {image}: {e}")  # Print the file causing the issue and skip
    features = np.array(features)
    features = features.reshape(features.shape[0], 128, 128, 3)  # Reshape with reduced size
    return features, valid_images

TRAIN_DIR = "/content/drive/MyDrive/New_Data"

train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

train_features, valid_images = extract_features(train['image'])

x_train = train_features / 255.0

# Update labels to match the valid images
train = train[train['image'].isin(valid_images)].reset_index(drop=True)

le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_train = to_categorical(y_train, num_classes=2)

model = Sequential()
# Convolutional layers with reduced filters
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))  # Reduced filters
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))  # Reduced filters
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512, activation='relu'))  # Reduced units
model.add(Dropout(0.3))
model.add(Dense(1024, activation='relu'))  # Reduced units
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train, y=y_train, batch_size=32, epochs=20, verbose=2)  # Increased batch size


Real completed
AI completed


  0%|          | 0/592 [00:00<?, ?it/s]

Skipping file /content/drive/MyDrive/New_Data/AI/image_326.jpg: cannot identify image file <_io.BytesIO object at 0x7c26c726c3b0>
Skipping file /content/drive/MyDrive/New_Data/AI/image_530.jpg: cannot identify image file <_io.BytesIO object at 0x7c26c7beafc0>


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
19/19 - 23s - 1s/step - accuracy: 0.5119 - loss: 0.8713
Epoch 2/20
19/19 - 20s - 1s/step - accuracy: 0.5559 - loss: 0.6807
Epoch 3/20
19/19 - 24s - 1s/step - accuracy: 0.5966 - loss: 0.6658
Epoch 4/20
19/19 - 38s - 2s/step - accuracy: 0.6983 - loss: 0.6294
Epoch 5/20
19/19 - 19s - 994ms/step - accuracy: 0.7441 - loss: 0.5630
Epoch 6/20
19/19 - 21s - 1s/step - accuracy: 0.7712 - loss: 0.5119
Epoch 7/20
19/19 - 20s - 1s/step - accuracy: 0.8305 - loss: 0.4217
Epoch 8/20
19/19 - 20s - 1s/step - accuracy: 0.8576 - loss: 0.3284
Epoch 9/20
19/19 - 19s - 1s/step - accuracy: 0.8797 - loss: 0.3042
Epoch 10/20
19/19 - 20s - 1s/step - accuracy: 0.9339 - loss: 0.1777
Epoch 11/20
19/19 - 19s - 1s/step - accuracy: 0.9356 - loss: 0.1612
Epoch 12/20
19/19 - 18s - 951ms/step - accuracy: 0.9593 - loss: 0.0983
Epoch 13/20
19/19 - 21s - 1s/step - accuracy: 0.9729 - loss: 0.0642
Epoch 14/20
19/19 - 19s - 992ms/step - accuracy: 0.9695 - loss: 0.0734
Epoch 15/20
19/19 - 19s - 1s/step - accuracy: 0.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Saving the model to a file
model.save('/content/drive/MyDrive/cnn_model(n).h5')  # 'cnn_model.h5' is the file where the model will be saved
# Saving just the weights
model.save_weights('/content/drive/MyDrive/model_weights(n).weights.h5')


In [ ]:
from keras.models import load_model

# # Load the saved model
model = load_model('/content/drive/MyDrive/cnn_model(n).h5')


In [ ]:
TEST_DIR='/content/drive/MyDrive/Test_Images'
def test_on_1_img(img_name):
    test_image_path = f"/content/drive/MyDrive/Test_Images/{img_name}.jpg"
    print(f"Testing image: {test_image_path}")  # Debugging line to verify the path

    try:
        img = load_img(test_image_path, target_size=(128, 128))  # Match the training size
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)  # Make it 4D
        img_array = img_array / 255.0  # Normalize image

        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)  # Get the class index with max probability
        predicted_label = le.inverse_transform(predicted_class)  # Convert index back to label
        prediction = predicted_label[0]  # Extract the predicted label

        return prediction

    except Exception as e:
        print(f"Error while processing image {img_name}: {e}")
        return None  # Return None if error occurs

def test(dir):
    image_name = []
    predicted_labels = []
    img_list = os.listdir(dir)  # No slicing, so all images will be processed

    for img_with_ext in img_list:
        img_name = img_with_ext[:-4]  # Remove .jpg or .png extension
        print(f"Processing image: {img_name}...")  # Debugging line to track processing

        predicted_label = test_on_1_img(img_name)

        if predicted_label is not None:
            predicted_labels.append(predicted_label)
            image_name.append(img_name)

    return image_name, predicted_labels

image_name, predicted_labels = test(TEST_DIR)
predicted_data = pd.DataFrame()
predicted_data['Id'], predicted_data['Label'] = image_name, predicted_labels

predicted_data.to_csv('final_prediction.csv', index=False)

predicted_data['Id'] = predicted_data['Id'].astype(str)  # Convert 'Id' column to string
predicted_data['number'] = predicted_data['Id'].str.extract('(\d+)').astype(int)
predicted_data = predicted_data.sort_values('number').drop('number', axis=1)

predicted_data
predicted_data.to_csv('final_prediction_sorted.csv',index=False)

Processing image: image_0...
Testing image: /content/drive/MyDrive/Test_Images/image_0.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Processing image: image_102...
Testing image: /content/drive/MyDrive/Test_Images/image_102.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Processing image: image_1...
Testing image: /content/drive/MyDrive/Test_Images/image_1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Processing image: image_106...
Testing image: /content/drive/MyDrive/Test_Images/image_106.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Processing image: image_10...
Testing image: /content/drive/MyDrive/Test_Images/image_10.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Processing image: image_104...
Testing image: /content/drive/MyDrive/Test_Images/image_104.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Processing image: image_105...
Testing image: /content/drive/MyDrive/Test_Images/image_105.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Processing image: image_103...
Testing image: /content/drive/MyDrive/Test_Image